## 1. Bayesian decision theory

> *对分类任务来说，在所有相关的概率已知的情况下，贝叶斯决策论会考虑概率和误判损失来选择最优的类别标记*

对情感分析来说，假设有N种情感，记为$C = \{c_1, c_2\, ... , c_N\}$，又记 $\lambda_{ij} $为将真实为 $c_j$ 的样本 **x** 分类成 $c_i$ 产生的损失<br>
则 $P(c_i \mid \mathbf{x})$ 为将样本分类为为$c_i$的 *expected loss* , 即为在此样本上产生的 **conditional risk**: 
$$ R(c_i \mid \mathbf{x}) = \sum_{j=1}^N\lambda_{ij}P(c_j \mid \mathbf{x}) $$ <br>
使得 *risk* 最小，即寻找 **Bayes optimal classifier** : $$ f^*(x) = arg \;min\,R(c\,|\,\mathbf{x})\ ,\quad c \in C $$ <br>
此时对应的总体风险为 $R(h^*)$, 称为**Bayes risk**。  $1-R(h^*)$反映了该分类器的最好*performance*, 是该机器学习算法精度的理论上限。
对应最简单的一种情况:$$\lambda=\begin{cases}
0& \text{i = j}\\
1& \text{i != j}
\end{cases}$$
则$R(c \mid \mathbf{x}) = 1 - P(c \mid \mathbf{x})$, 最优的分类器：$$ f^*(x) = arg \;max\,P(c\,|\,\mathbf{x})\ ,\quad c \in C $$ <br>
所以，要获得最优分类器，首先要获得 **后验概率(posterior probability)** ：$P(c \mid \mathbf{x})$ ，通过著名的 ***贝叶斯公式*** :<br><br> $$ P(c \mid \mathbf{x}) = \frac{P(\mathbf{x} \mid c)P(c)} {P(\mathbf{x})} $$

$P(c)$ 被称为**先验概率(prior probability)**, 在情感分析中，具体代表某类情感出现的概率，根据大数定律，样本集足够大时，可通过统计频率来进行估计。 <br>$P(\mathbf{x})$只是用来归一化的**证据(evidence)** ，显然可以从样本中 $\mathbf{x}$所占频率得出。<br>$ P(\mathbf{x} \mid c) $ 是样本 $\mathbf{x}$ 相对于类别 c 的**类条件概率(class-conditional probabilty)** ，也称为**似然(likelihood)** 。<br>
真正的难点来了！一个样本 $\mathbf{x}$ 往往会包含多种属性(一句话通常会抽象为多个特征组成的向量)，$P(\mathbf{x} \mid c)$ 牵涉到$\mathbf{x}$的所有属性的联合概率。假设样本有$n$个属性，就算样本每个属性都是二值性，样本空间中也会包含$2^n$种，这可是指数级的! 一般会远大于样本集数量 $m$ ,所以如果要通过频率去估计，显然是错误的，因为未出现的样本我们没有考虑进去，而它们概率很多时候都不为0

## 2.Maximun Likelihood Estimation

> *概率模型的训练过程就是**参数估计(parameter estimation)**的过程, 而关于参数估计，统计学界的两个学派争执不休：MLE属于频率主义学派(Frequentist)，他们认为参数虽然未知，但却是客观存在的固定值，通过不断优化似然函数可得到这个值。而贝叶斯学派(Bayesian)认为参数是未观察到的随机变量，本身可有分布。假定参数服从一个先验分布，后基于观测到的数据来计算参数的后验分布*

MLE绝对属于根据数据样本估计概率分布的经典方法: <br>
&emsp;&emsp;令 $D_c$ 表示训练集 $D$ 中第 $c$ 类样本集合, 假设样本是**独立同分布**的(*所以数据集的分布很重要 !!*)，则我们可以估计一个参数 $\theta_c$ ，对于数据集 $D_c $的似然：<br><br>$$ P(D_c \mid \theta_c) = \prod_{ \mathbf{x} \in D_c} P(\mathbf{x} \mid \theta_c) $$
我们要做的，就是寻找一个$\theta$使得上面的$P(D_c \mid \theta_c)$, 也就是使这个数据集出现的概率最大。<br>
因为概率连乘在计算机中容易造成下溢，所以通常使用**对数似然(log-likelihood)**：<br><br>
$$\begin{eqnarray*} LL(\theta_c) &=& log \ P(D_c \,|\, \theta_c) \\ &=& \sum_{\mathbf{x} \in D_c}logP(\mathbf{x} \mid \theta_c) \end{eqnarray*}$$

$\theta$ 的极大似然估计 $\hat\theta_c$ ：$$ \hat\theta_c = arg \ max(LL(\theta_c)) $$
一种偷懒的做法：如果样本属性连续，假设概率密度函数$p(x) \sim N(\mu_c, \sigma_c^2) $, 可得出$\mu_c$和$\sigma_c$的极大似然估计 :<br>
$$ \hat\mu_c = \frac{1}{\left | D_c \right |}\sum_{x \in D_c}x $$ 
$$ \hat\sigma^2 = \frac{1}{\left | D_c \right |}\sum_{x \in D_c}(x - \hat\mu_c)(x - \hat\mu_c)^T$$
其中$\left | D_c \right |$代表数据集$D_c$的样本数<br>
这种方法的痛点在于要求数据的概率分布是根据直觉猜出来的，是否符合真实数据的分布无法得知, 所以也许会产生误导性结果

## 3. naive Bayes Classifier

可以看出，贝叶斯的扩展式实际是这样的：
\begin{aligned}
P(C|F_1F_2\cdots F_n) & =\frac{P(C)\cdot P(F_1F_2\cdots F_n \mid C)}{P(F_1F_2\cdots F_n)} \\
& =\frac{P(C)\cdot P(F_1\mid C) \cdot P(F_2\cdots F_n \mid CF_1)}{P(F_1F_2\cdots F_n)} \\
& =\cdots \\
& =\frac{P(C)\cdot P(F_1\mid C) \cdot P(F_2\mid CF_1)\cdots P(F_n \mid CF_1F_2\cdots F_{n-1})}{P(F_1F_2\cdots F_n)}
\end{aligned}
其中$F_i$是样本x的属性，其实等于$x_i$，这里用F是为了让大家更直观体会到它是 *"Feature"*。而每一个$P(F_i \cdots F_n \mid CF_1F_{i-1})$都是一个似然！<br>

为了简化复杂的计算，朴素贝叶斯假设样本的各个属性间都是独立的，这样复杂的式子变得"清新"不少：
\begin{aligned}
P(C|F_1F_2\cdots F_n) & =\frac{P(C)\cdot P(F_1F_2\cdots F_n \mid C)}{P(F_1F_2\cdots F_n)} \\
& =\frac{P(C)\cdot P(F_1\mid C) \cdot P(F_2\mid C)\cdots P(F_n \mid C)}{P(F_1F_2\cdots F_n)}
\end{aligned}
即$$P(c \mid \mathbf{x}) = \frac{P(c)}{P(x)}\prod_{i=1}^n P(x_i \mid c) $$
所以朴素贝叶斯分类器：
$$ h_{nb}(\mathbf{x}) = arg\ max P(c)\prod_{i=1}^nP(x_i \mid c)\ \ , c \in C $$
要计算出这个表达式，$P(c) = \frac{\left | D_c \right |}{D}$ ，如果$\mathbf{x}$的属性都是离散属性，那么
$$P(x_i \mid c) = \frac{\left | D_{c,x_i} \right |}{\left | D_c \right |}$$
如果$x_i$是连续属性，那么考虑概率密度函数, 假定$p(x_i \mid c) \sim N(\mu_c, \sigma_{c,i}^2)$, 其中$\mu_c$和$\sigma_{c,i}^2$是样本属性i在第c类上的均值和方差，则有
$$ p(x_i \mid c) = \frac{1}{\sqrt{2\pi}\sigma_{c,i}}exp \left ( -\frac{(x_i - \mu_{c,i})^2}{2\sigma_{c,i}^2} \right ) $$
不过这里存在一个严重的问题，如果测试时一个属性的值在训练集从未出现过，这样计算出来的$P(x_i \mid c)$就为0，连乘之后，导致$\prod_{i=1}^n P(x_i \mid c)$整个为0！！<br>
我们显然不希望这样，它代表完全忽略新属性值出现的可能性，这样的学习器就好像一个书呆子只会做书本上的题一样连一点悟性都没有，所以，在估计概率值时通常会进行 **"平滑(smoothing)"** ，常用的是**拉普拉斯修正(Laplacian correction)**：
$$ \hat P(c) = \frac{\left | D_c \right |\ +\ 1}{\left | D \right | \ +\ N} $$ <br>
$$ \hat P(x_i \mid c) = \frac{\left | D_{c,x_i} \right |\ +\ 1}{\left | D_c \right | \ +\ N_i} $$

其中$N$代表训练集$D$中可能的类别数，$N_i$表示第i个属性可能的取值数。分母加了$N_i$是很好玩的，最理想的情况它将使互斥事件的概率改变后相加还是1。当然$N$， $N_i$都是我们自己设定的参数。这时就需要对这个数据集本身有所理解了。<br>
关于使用贝叶斯分类其实有多种trick，比如采用**懒惰学习(lazy learning)** ，接收到预测请求时再根据当前数据集训练；采用储存概率表，可在计算时进行**查表** ，对提速很有用; 数据增加时，对新样本的属性牵涉的概率进行计数修正可实现**增量学习(Incremental learning)** <br>

**总结**一下使用朴素贝叶斯分类器的情况吧，需要我们选择的地方：假设这个连续属性的分布是*正态分布*还是其它的分布如*伯努利分布*等，第二就是对平滑的选择方法，大多时候使用*拉普拉斯平滑（据说是他在看日出的时候通过想明天太阳是否升起想出来的）* ，当然还有其它的平滑方案如**Lidstone平滑**(通过增加一个大于0的可调参数$\alpha$)，然后是选择平滑的参数。所以，目前的我感觉无论是什么学习算法，最终都会来到这一步：对数据集本身的理解

## 4.Sentiment classification in action
深刻理解一个学习算法，除了推出公式外，当然还要进行实战啊，下面是我用python写的demo。<br>
数据来源是康奈尔大学网站的2M影评数据集，可在http://download.csdn.net/detail/lsldd/9346233 进行下载
